## Deliverable 2. Create a Customer Travel Destinations Map.

In [5]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,55.51,41,20,20.71,few clouds
1,1,Blackwater,AU,-23.5833,148.8833,76.57,72,67,12.48,broken clouds
2,2,Khatanga,RU,71.9667,102.5000,-22.94,100,99,6.76,overcast clouds
3,3,Lapithos,CY,35.3375,33.1792,58.23,67,0,5.14,clear sky
4,4,Mar Del Plata,AR,-38.0023,-57.5575,63.88,77,11,15.01,few clouds


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Blackwater,AU,-23.5833,148.8833,76.57,72,67,12.48,broken clouds
6,6,Georgetown,MY,5.4112,100.3354,77.90,77,20,10.36,light rain
11,11,Atuona,PF,-9.8000,-139.0333,78.49,75,5,18.30,clear sky
16,16,Vaini,TO,-21.2000,-175.2000,80.76,89,75,13.80,broken clouds
17,17,Kapaa,US,22.0752,-159.3190,79.48,71,40,3.00,scattered clouds
20,20,Arraial Do Cabo,BR,-22.9661,-42.0278,75.16,88,20,16.11,few clouds
26,26,Saint-Georges,GD,12.0564,-61.7485,78.48,89,20,12.66,few clouds
27,27,Lorengau,PG,-2.0226,147.2712,85.08,75,60,16.35,broken clouds
44,44,Hithadhoo,MV,-0.6000,73.0833,81.23,78,100,12.95,overcast clouds
50,50,Calabar,NG,4.9517,8.3220,77.49,91,99,3.44,overcast clouds


In [10]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             189
City                   189
Country                189
Lat                    189
Lng                    189
Max Temp               189
Humidity               189
Cloudiness             189
Wind Speed             189
Current Description    189
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = city_data_df.dropna()
clean_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,55.51,41,20,20.71,few clouds
1,1,Blackwater,AU,-23.5833,148.8833,76.57,72,67,12.48,broken clouds
2,2,Khatanga,RU,71.9667,102.5000,-22.94,100,99,6.76,overcast clouds
3,3,Lapithos,CY,35.3375,33.1792,58.23,67,0,5.14,clear sky
4,4,Mar Del Plata,AR,-38.0023,-57.5575,63.88,77,11,15.01,few clouds


In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Punta Arenas,CL,55.51,few clouds,-53.1500,-70.9167,
1,Blackwater,AU,76.57,broken clouds,-23.5833,148.8833,
2,Khatanga,RU,-22.94,overcast clouds,71.9667,102.5000,
3,Lapithos,CY,58.23,clear sky,35.3375,33.1792,
4,Mar Del Plata,AR,63.88,few clouds,-38.0023,-57.5575,
5,Albany,US,37.81,overcast clouds,42.6001,-73.9662,
6,Georgetown,MY,77.90,light rain,5.4112,100.3354,
7,Leningradskiy,RU,-3.14,snow,69.3833,178.4167,
8,Meadow Lake,US,65.17,clear sky,34.8014,-106.5436,
9,Cape Town,ZA,62.10,overcast clouds,-33.9258,18.4232,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [19]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   662
Country                662
Max Temp               662
Current Description    662
Lat                    662
Lng                    662
Hotel Name             662
dtype: int64

In [20]:
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Punta Arenas,CL,55.51,few clouds,-53.1500,-70.9167,Hotel Hain
1,Blackwater,AU,76.57,broken clouds,-23.5833,148.8833,Blackwater Motor Inn
2,Khatanga,RU,-22.94,overcast clouds,71.9667,102.5000,"Gostinitsa ""Zapolyar'ye"""
3,Lapithos,CY,58.23,clear sky,35.3375,33.1792,Gillham Luxury Boutique & Vineyard
4,Mar Del Plata,AR,63.88,few clouds,-38.0023,-57.5575,Gran Hotel Mar del Plata


In [22]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [43]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [44]:
# Get the maximum temperature.
max_temp = clean_hotel_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [45]:
#11a.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=temps,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
#11b.
fig

Figure(layout=FigureLayout(height='420px'))